In [1]:
from appgeopy import *
from my_packages import *

In [2]:
# Load Persistent Scatterer (PS) displacement data from a pickle file
psc_fpath = r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_004\14_POST-PROCESSING\ras2pnt_MODIFIED_oChoushui_DISPLACEMENT_LOS_mm.pkl"
psc_gdf = pd.read_pickle(psc_fpath, compression="zip")
psc_gdf.set_index("PointKey", inplace=True)

# Extract column names representing dates
datetime_columns = [col for col in psc_gdf.columns if col.startswith("20")]

# Load the PointKey selection for PS points surrounding GPS stations
select_pointkey_fpath = r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PS_surroundGPS2023_PointKey_1000m.pkl"
select_pointkey = pd.read_pickle(select_pointkey_fpath, compression="zip")

# Load the GPS LOS difference data from an Excel file
gps_los_difference_fpath = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2023_YehDaGang\2023_NEW_20240515\20240527_PROCESSED_DATA\_MERGED\6__Difference_Modeled\Modeled_GPS_Differences_LOS_mm_2022_2023_v1.xlsx"

# Get the unique GPS stations from the selection
available_stations = select_pointkey["STATION"].unique()

# Select the first available station for analysis
select_station = available_stations[0]

# Filter the PointKeys associated with the selected station
pointkey_by_station = select_pointkey.loc[
    select_pointkey["STATION"] == select_station, "PointKey"
]

# Extract the displacement data for the selected station
psc_by_station = psc_gdf.loc[pointkey_by_station, datetime_columns]

# Calculate the average displacement over all PS points for the selected station
avg_deform_psc_by_station = pd.DataFrame(
    psc_by_station.mean(axis=0), columns=["LOS(mm)"]
)
avg_deform_psc_by_station.index = datetime_handle.convert_to_datetime(
    avg_deform_psc_by_station.index
)

# Load the GPS LOS difference data for the selected station
gps_los_difference_by_station = pd.read_excel(
    gps_los_difference_fpath,
    sheet_name=select_station,
    usecols=[
        0,
        4,
    ],  # Assume the first column is date and the fifth column is LOS difference
    parse_dates=[0],  # Parse the first column as dates
    index_col=[0],  # Set the first column as the index
)

# Find the common dates between PS and GPS data
mutual_index = datetime_handle.intersect_time_index(
    df1_index=avg_deform_psc_by_station.index,
    df2_index=gps_los_difference_by_station.index,
)

# Compare LOS displacements between PS and GPS data over the common dates
los_difference = ts_disp.compare_LOS_disp(
    psc_df=avg_deform_psc_by_station,
    gps_df=gps_los_difference_by_station,
    mutual_index=mutual_index,
)

# data_io.save_df_to_excel()

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\030_CHOUSHUI_2024\\000_INSCALDEFO_2_INSSTACKPSI\\PROCESS_003\\ras2pnt_oChoushui_DISPLACEMENT_LOS_mm.pkl'

In [ ]:
los_difference